# Generate Toyota Prius Motor from script

In [ ]:
from pyleecan.Classes.LamSlotWind import LamSlotWind
from pyleecan.Classes.LamSlotMag import LamSlotMag
from pyleecan.Classes.LamSlot import LamSlot
from pyleecan.Classes.Winding import Winding
from pyleecan.Classes.SlotW10 import SlotW10
from pyleecan.Classes.SlotW22 import SlotW22
from pyleecan.Classes.Shaft import Shaft
from pyleecan.Classes.Material import Material
from pyleecan.Classes.MachineIPMSM import MachineIPMSM
from pyleecan.Classes.LamHole import LamHole
from pyleecan.Classes.HoleM50 import HoleM50
from pyleecan.Classes.Magnet import Magnet
from pyleecan.Classes.MatMagnetics import MatMagnetics
from pyleecan.Functions.load import load
from pyleecan.definitions import DATA_DIR
from pyleecan.Classes.MatElectrical import MatElectrical
from pyleecan.Classes.ImportMatrix import ImportMatrix
from pyleecan.Classes. MatStructural import MatStructural
from pyleecan.Classes.ModelBH import ModelBH
from pyleecan.Classes.MatHT import MatHT
from pyleecan.Classes.CondType12 import CondType12
from pyleecan.Classes.SlotW11 import SlotW11
from pyleecan.Classes.EndWinding import EndWinding
from pyleecan.Classes.Simu1 import Simu1
from pyleecan.Classes.Electrical import Electrical
from pyleecan.Classes.EEC_PMSM import EEC_PMSM
from pyleecan.Classes.MagFEMM import MagFEMM
from pyleecan.Classes.InputCurrent import InputCurrent
from pyleecan.Classes.OPdq import OPdq
from pyleecan.Functions.Plot import dict_2D, dict_3D

import numpy as np
from numpy import sqrt, exp, pi
from os.path import join
from util.simulation import *
from util.failures import *
import matplotlib.pyplot as plt
import itertools
from deepdiff import DeepDiff

from util.toyota_prius_generator import *

## Toyota Prius with default parameters

In [ ]:
tpg = Toyota_Prius_Generator()
shaft = tpg.create_shaft()
rotor = tpg.create_rotor()
stator = tpg.create_stator()
#create machine
machine = MachineIPMSM(
    name="Toyota Prius",
    shaft=shaft,
    rotor=rotor,
    stator=stator,
    type_machine = 1
)

fig, ax = machine.plot(is_show_fig=False)

In [ ]:
simu_femm = load_simulation(name="Prius", machine=machine, rotor_speed=3000)
out_femm = run_simulation(simu_femm)
plot_simulation_results(out_femm)

## Create a Toyota Prius with failure

In [ ]:
#create machine with failure
machine_with_failure = create_machine_with_winding_failure(generator=tpg, Ntcoil = 1)

simu_femm_failure = load_simulation(name="Prius", machine=machine_with_failure, rotor_speed=3000)

out_femm_failure = run_simulation(simu_femm_failure)
compare_simulation_results(out1=out_femm, out2=out_femm_failure)

In [ ]:
tpg = Toyota_Prius_Generator()
for i in range (1, 9):
    machine_with_failure = create_machine_with_winding_failure(generator=tpg, Ntcoil = i)
    simu_femm_failure = load_simulation(name="Prius", machine=machine_with_failure, rotor_speed=3000)
    out_femm_failure = simu_femm_failure.run()
    # Tangential magnetic flux
    out_femm.mag.B.plot_2D_Data("time", component_list=["tangential"], data_list=[out_femm_failure.mag.B], legend_list=["Reference", "Winding failure "+str(i)], **dict_2D)

## Change winding

In [ ]:
tpg.Ntcoil = 100
shaft = tpg.create_shaft()
rotor = tpg.create_rotor()
stator = tpg.create_stator()
#create machine
machine = MachineIPMSM(
    name="Toyota Prius",
    shaft=shaft,
    rotor=rotor,
    stator=stator,
    type_machine = 1
)
simu_femm = load_simulation(name="Prius", machine=machine, rotor_speed=3000)
out_femm = run_simulation(simu_femm)
#create machine with failure
machine_with_failure = create_machine_with_winding_failure(generator=tpg, Ntcoil = 1)

simu_femm_failure = load_simulation(name="Prius", machine=machine_with_failure, rotor_speed=3000)

out_femm_failure = run_simulation(simu_femm_failure)
compare_simulation_results(out1=out_femm, out2=out_femm_failure)

## Change magnet material

In [ ]:
tpg.Ntcoil = 9
tpg.magnet_material = load(join(DATA_DIR, "Material", "Magnet_50SH.json"))
#create machine
machine = MachineIPMSM(
    name="Toyota Prius",
    shaft=tpg.create_shaft(),
    rotor=tpg.create_rotor(),
    stator=tpg.create_stator(),
    type_machine = 1
)
simu_femm_diff_mag = load_simulation(name="Prius", machine=machine, rotor_speed=3000)
out_femm_diff_mag = run_simulation(simu_femm_diff_mag)
compare_simulation_results(out1=out_femm, out2=out_femm_diff_mag)

In [ ]:
# Main geometry parameter
Rext = 170e-3  # Exterior radius of outter lamination
W1 = 30e-3  # Width of first lamination
A1 = 2.5e-3  # Width of the first airgap
W2 = 20e-3

## GMSH export

In [ ]:
gmsh_export(out=out_femm, path_save="prius.msh")

In [ ]:
Electric simulation

In [ ]:
# Initialization of the Simulation
simu = Simu1(name="tuto_elec", machine=machine)
# Definition of the Electrical Equivalent Circuit
eec = EEC_PMSM(  # No parameter enforced => compute all
    fluxlink=MagFEMM(is_periodicity_a=False, T_mag=60),  # Magnetic model to compute fluxlinkage
)
# The Electrical module is defined with the EEC
simu.elec = Electrical(eec=eec)

# Run only Electrical module
simu.mag = None
simu.force = None
simu.struct = None

In [ ]:
# Definition of a sinusoidal current
simu.input = InputCurrent()

I0_rms = 250/sqrt(2) # Maximum current [Arms]
Phi0 = 140*pi/180  # Maximum Torque Per Amp

simu.input.OP = OPdq(N0 = 2000)
simu.input.OP.set_I0_Phi0(I0_rms, Phi0)

simu.input.Nt_tot = 50 # Number of time steps
simu.input.Na_tot = 2048 # Spatial discretization

(simu.input.OP.Id_ref,simu.input.OP.Iq_ref)

In [ ]:
out = simu.run()

In [ ]:
# Print voltage and torque
print("Ud: "+str(out.elec.OP.Ud_ref))
print("Uq: "+str(out.elec.OP.Uq_ref))
print("Tem: "+str(out.elec.Tem_av))

# Electrical Equivalent Circuit parameters are stored in:
print("\nLd: "+str(out.elec.eec.Ld))
print("Lq: "+str(out.elec.eec.Lq))
print("R1: "+str(out.elec.eec.R1))

# Plot the currents
out.elec.get_Is().plot_2D_Data("time", "phase")

# Plot the voltages
out.elec.get_Us().plot_2D_Data("time", "phase")

##  Electrical + Magnetic simulation 

In [ ]:
simu.mag = simu_femm.mag

In [ ]:
out = simu.run()

In [ ]:
# Radial magnetic flux
out.mag.B.plot_2D_Data("angle","time[1]",component_list=["radial"], is_show_fig=False)
out.mag.B.plot_2D_Data("freqs",component_list=["radial"], is_show_fig=False)
# Tangential magnetic flux
out.mag.B.plot_2D_Data("angle","time[1]",component_list=["tangential"], is_show_fig=False)
out.mag.B.plot_2D_Data("freqs",component_list=["tangential"], is_show_fig=False)

In [ ]:
machine.rotor.build_geometry(sym=1, delta=complex(0.002, 0.005), is_circular_radius=False)
fig, ax = machine.plot(is_show_fig=False)

In [ ]:
for i in range (1, 20):
    tpg.Ntcoil = i
    machine.stator = tpg.create_stator()
    simu_femm = load_simulation(name="Prius", machine=machine, rotor_speed=3000, stop=0.02, num_steps=256)
    out_femm = simu_femm.run()
    # Tangential magnetic flux
    out_femm.mag.B.plot_2D_Data("angle","time[1]",component_list=["tangential"], is_show_fig=False)
    out_femm.mag.B.plot_2D_Data("freqs",component_list=["tangential"], is_show_fig=False)
